In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from dataloading.loaders.openreview_loader import OpenReviewLoader

In [26]:
password = 'U^fz&*rXwXr8pyx'
username = 'zhijingshui.yang@stud.tu-darmstadt.de'
try:
    client = OpenReviewLoader(username=username, password=password)
except Exception as e:
    print(f"{e.args[0]['status']}: {str(e.args[0]['message'])}")
    print(e)

test_set = client.create_testset(ids=["zzv4Bf50RW"])

2024-12-01 23:22:40.892 | INFO     | dataloading.loaders.openreview_loader:__init__:31 - OpenReview client initialized.
2024-12-01 23:22:49.733 | INFO     | dataloading.loaders.openreview_loader:_get_reviews_from_multiple_papers:178 - Fetching reviews from multiple papers
2024-12-01 23:22:49.734 | INFO     | dataloading.loaders.openreview_loader:get_paper:69 - Fetching paper with id: zzv4Bf50RW
2024-12-01 23:22:50.111 | INFO     | dataloading.loaders.openreview_loader:get_paper:87 - Fetched paper: Learning SO(3)-Invariant Correspondence via Point-wise Local Shape Transform
2024-12-01 23:22:50.111 | INFO     | dataloading.loaders.openreview_loader:_get_reviews:160 - Getting reviews for paper: Learning SO(3)-Invariant Correspondence via Point-wise Local Shape Transform
2024-12-01 23:22:50.112 | INFO     | dataloading.loaders.openreview_loader:prepare_review:113 - Preparing review for note: aU4SaZOeh4
2024-12-01 23:22:50.113 | INFO     | dataloading.loaders.openreview_loader:prepare_revie

In [28]:
p = client.get_paper_reviews("zzv4Bf50RW")
p.reviews
print(p.sentences)

2024-12-01 23:27:05.196 | INFO     | dataloading.loaders.openreview_loader:get_paper:69 - Fetching paper with id: zzv4Bf50RW
2024-12-01 23:27:05.740 | INFO     | dataloading.loaders.openreview_loader:get_paper:87 - Fetched paper: Learning SO(3)-Invariant Correspondence via Point-wise Local Shape Transform
2024-12-01 23:27:05.741 | INFO     | dataloading.loaders.openreview_loader:_get_reviews:160 - Getting reviews for paper: Learning SO(3)-Invariant Correspondence via Point-wise Local Shape Transform
2024-12-01 23:27:05.742 | INFO     | dataloading.loaders.openreview_loader:prepare_review:113 - Preparing review for note: aU4SaZOeh4
2024-12-01 23:27:05.743 | INFO     | dataloading.loaders.openreview_loader:prepare_review:113 - Preparing review for note: wKVoAVtxek
2024-12-01 23:27:05.744 | INFO     | dataloading.loaders.openreview_loader:prepare_review:113 - Preparing review for note: NLfuL11uwO
2024-12-01 23:27:05.744 | INFO     | dataloading.loaders.openreview_loader:prepare_review:113

           author                                           sentence
0   Reviewer eS3u  This work proposes LSTNet, a self-supervised m...
1   Reviewer eS3u  Specifically, LSTNet learns to formulate SO(3)...
2   Reviewer eS3u  The proposed self-supervised training pipeline...
3   Reviewer eS3u  The self- and cross-reconstruction training st...
4   Reviewer eS3u  LSTNet demonstrates state-of-the-art performan...
5   Reviewer eS3u  The performance of aligned shape pairs under t...
6   Reviewer eS3u  The reason why other methods are much better t...
7   Reviewer eS3u                               Lack of limitations.
8   Reviewer jP4i  1) This paper proposes a self-supervised metho...
9   Reviewer jP4i  2）The main idea is to decouple a point cloud f...
10  Reviewer jP4i  3) Experiments on the KeypointNet dataset show...
11  Reviewer jP4i           1) This paper is generally well-written.
12  Reviewer jP4i  2) The idea of factorizing point cloud descrip...
13  Reviewer jP4i                 

In [3]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('models/attitude_root/')

# Load the model
model = TFBertForSequenceClassification.from_pretrained('models/attitude_root/')

2024-12-01 21:02:50.715957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733086970.730716   15876 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733086970.736163   15876 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 21:02:50.753642: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/nana/anaconda3/envs/jitsupeer_310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not foun

In [4]:
def predict_category(text):
    predict_input = tokenizer.encode(text,
                                    truncation=True,
                                    padding=True,
                                    return_tensors="tf")
    output = model(predict_input)[0]
    prediction_value = tf.argmax(output, axis=1).numpy()[0]
    return prediction_value

In [8]:
test_set['attitude_root_number'] = test_set['sentence'].apply(predict_category)
print(test_set)

           author                                           sentence  \
0   Reviewer eS3u  This work proposes LSTNet, a self-supervised m...   
1   Reviewer eS3u  Specifically, LSTNet learns to formulate SO(3)...   
2   Reviewer eS3u  The proposed self-supervised training pipeline...   
3   Reviewer eS3u  The self- and cross-reconstruction training st...   
4   Reviewer eS3u  LSTNet demonstrates state-of-the-art performan...   
5   Reviewer eS3u  The performance of aligned shape pairs under t...   
6   Reviewer eS3u  The reason why other methods are much better t...   
7   Reviewer eS3u                               Lack of limitations.   
8   Reviewer jP4i  1) This paper proposes a self-supervised metho...   
9   Reviewer jP4i  2）The main idea is to decouple a point cloud f...   
10  Reviewer jP4i  3) Experiments on the KeypointNet dataset show...   
11  Reviewer jP4i           1) This paper is generally well-written.   
12  Reviewer jP4i  2) The idea of factorizing point cloud descri

In [ ]:
label_mapping = {
    0: 'Other',
    1: 'Clarity',
    2: 'Meaningful-comparison',
    3: 'Motivation-impact',
    4: 'Originality',
    5: 'Replicability',
    6: 'Soundness-correctness',
    7: 'Substance',
    8: 'None'
}

test_set['attitude_root'] = test_set['attitude_root_number'].map(label_mapping)

In [23]:
import pandas as pd

df = test_set
total_rows = len(df)
# Step 1: Group by 'attitude_root_number' and 'author', then aggregate the sentences
agg_df = df.groupby(['attitude_root_number', 'author']).agg(
    comments=('sentence', list), count=('sentence', 'size')
).reset_index()

agg_df


,attitude_root_number,author,comments,count
0,0,Reviewer Frem,[1. The paper is in general well organized and...,3
1,0,Reviewer a6Ps,"[Valid motivation. Unlike the abused topic, va...",6
2,0,Reviewer eS3u,"[Specifically, LSTNet learns to formulate SO(3...",2
3,0,Reviewer jP4i,"[1) This paper is generally well-written., 2) ...",3
4,0,Reviewer wiS9,[2. The overall writing is good and the method...,8
5,6,Reviewer Frem,[This paper presents a method of learning dens...,5
6,6,Reviewer a6Ps,[This paper attempts to register point cloud p...,8
7,6,Reviewer eS3u,"[This work proposes LSTNet, a self-supervised ...",6
8,6,Reviewer jP4i,[1) This paper proposes a self-supervised meth...,8
9,6,Reviewer wiS9,"[This paper introduces LSTNet, which leverages...",7


In [ ]:
final_df = agg_df.groupby('attitude_root_number').agg(
    comments=('comments', lambda x: [[author, comments] for author, comments in zip(agg_df['author'], x)]),
    count=('count', 'sum')
).reset_index()
final_df.iloc[0]['comments']

,attitude_root_number,comments,count
0,0,"[[Reviewer Frem, [1. The paper is in general w...",22
1,6,"[[Reviewer Frem, [This paper presents a method...",34


In [25]:
final_df.iloc[0]['comments']


[['Reviewer Frem',
  ['1. The paper is in general well organized and easy to follow.',
   '2. The proposed method is straightforward and shown to be effective on the test data.',
   "4. The whole method is mainly built upon the existing SO(3)-equivariant representation. The main contribution lies in introducing this representation to the specific task. I didn't get too much novel insight in terms of network design."]],
 ['Reviewer a6Ps',
  ['Valid motivation. Unlike the abused topic, vanilla point cloud registration, the motivation stands and could potentially benefit practical usages.',
   'The SO(3)-invariant network design intrinsically ensures robustness against rotations.',
   'In motivation, the authors talk about usage in vision, graphics, and robotics. In vision and robotics, we are interested in fitting real-world scans to templates (e.g. [Scan2CAD, CVPR 2019]), where in most cases, only noisy, partial, and sparse point clouds are provided. The authors do not have experiments 